In [1]:
# using http://www.easy-tensorflow.com/autoencoders/noise-removal

In [2]:
# https://github.com/aymericdamien/TensorFlow-Examples/blob/master/notebooks/3_NeuralNetworks/autoencoder.ipynb

In [1]:
!module list 

Currently Loaded Modulefiles:
  1) cuda-toolkit/9.0.176   2) cuDNN/9.0v7


In [1]:
import numpy as np
import scipy.sparse as ss
import tensorflow as tf
import matplotlib.pyplot as plt
import pickle

/home/esadrfa/libs/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
featurePath = '/scratch2/esadrfa/perceptron/data/'

In [3]:
# %%time

# csr_mat = pickle.load(open(featurePath + 'SO_CS_EU_Transposed_scaled_s0_e99999_csr.p', 'rb'))

#### I saved the transposed matrix for better performance on SVD but now I am using autoencoder.
#### Hence, I have to transpose it again to get each data point as a row 

In [4]:
# csr_mat_100k_23M = csr_mat.transpose()

# csr_mat_100k_23M

In [5]:
# chunk_size = 20000
# for i in range(5):
#     s = chunk_size * i
#     small_mat = csr_mat_100k_23M[s: s + chunk_size]
#     print(s, s + chunk_size)
#     pickle.dump(small_mat, open(featurePath \
#                      + 'SO_CS_EU_scaled_s0_e99999_csr_part'+ str(i) +'.p', 'wb'), protocol=4)


In [6]:
%%time
train = pickle.load(open(featurePath + 'SO_CS_EU_scaled_s0_e99999_csr_part'+ '4' +'.p', 'rb'))

CPU times: user 2.97 s, sys: 7.84 s, total: 10.8 s
Wall time: 10.8 s


In [7]:
%%time
data = train.tocsr()

CPU times: user 20.6 s, sys: 3.62 s, total: 24.2 s
Wall time: 24.3 s


In [8]:
type(data)

scipy.sparse.csr.csr_matrix

In [9]:
assert(type(data)!='scipy.sparse.csr.csr_matrix'), "the matrix is not csr"

In [10]:
%%time
test = pickle.load(open(featurePath + 'SO_CS_EU_scaled_s0_e99999_csr_part'+ '3' +'.p', 'rb'))

CPU times: user 3.46 s, sys: 9.03 s, total: 12.5 s
Wall time: 12.5 s


In [11]:
%%time
test = test.tocsr()

CPU times: user 23 s, sys: 4.12 s, total: 27.1 s
Wall time: 27.2 s


In [13]:
assert(type(test)!='scipy.sparse.csr.csr_matrix'), "the matrix is not csr"

In [14]:
load_batch_size = 100

In [15]:
# %%time
# coo_matrix = A[:load_batch_size].tocoo()
# print('number of rows:' ,coo_matrix.row)

# tf_coo_matrix = tf.SparseTensorValue(
#     indices=np.array([coo_matrix.row, coo_matrix.col]).T,
#     values=coo_matrix.data,
#     dense_shape=coo_matrix.shape)

# print('type:', type(tf_coo_matrix), '\nshape:', tf_coo_matrix.dense_shape)

In [12]:
# hyper-parameters
logs_path = "./logs/noiseRemoval_test_csr_matrix"  # path to the folder that we want to save the logs for Tensorboard
learning_rate = 0.001  # The optimization learning rate
epochs = 10  # Total number of training epochs
batch_size = 100  # Training batch size
display_freq = 100  # Frequency of displaying the training results

# Network Parameters
# We know that MNIST images are 28 pixels in each dimension.
# img_h = img_w = 100
num_features = data.shape[1]

# Images are stored in one-dimensional arrays of this length.
# img_size_flat = img_h * img_w

# number of units in the hidden layer
h1 = 1000

# level of the noise in noisy data
# noise_level = 0.6

In [13]:
print(num_features)

23058100


In [14]:
# weight and bais wrappers
def weight_variable(name, shape):
    """
    Create a weight variable with appropriate initialization
    :param name: weight name
    :param shape: weight shape
    :return: initialized weight variable
    """
    initer = tf.truncated_normal_initializer(stddev=0.01)
    return tf.get_variable('W_' + name,
                           dtype=tf.float32,
                           shape=shape,
                           initializer=initer)

def bias_variable(name, shape):
    """
    Create a bias variable with appropriate initialization
    :param name: bias variable name
    :param shape: bias variable shape
    :return: initialized bias variable
    """
    initial = tf.constant(0., shape=shape, dtype=tf.float32)
    return tf.get_variable('b_' + name,
                           dtype=tf.float32,
                           initializer=initial)

def fc_layer(x, num_units, name, use_relu=True):
    """
    Create a fully-connected layer
    :param x: input from previous layer
    :param num_units: number of hidden units in the fully-connected layer
    :param name: layer name
    :param use_relu: boolean to add ReLU non-linearity (or not)
    :return: The output array
    """
    with tf.variable_scope(name):
        in_dim = x.get_shape()[1]
        W = weight_variable(name, shape=[in_dim, num_units])
        tf.summary.histogram('W', W)
        b = bias_variable(name, [num_units])
        tf.summary.histogram('b', b)
        layer = tf.matmul(x, W)
        layer += b
        if use_relu:
            layer = tf.nn.relu(layer)
        return layer

##### convert the SparseTensorValue to SparseTensor

In [ ]:
# %%time
# # tf_coo_matrix
# st_matrix = tf.SparseTensor.from_value(tf_coo_matrix)
# # print(type(st_matrix))
# # curr_dense_matrix = tf.sparse_tensor_to_dense(st_matrix)
# # type(curr_dense_matrix)
# # curr_dense_matrix.shape

In [29]:
# import math
# num_train = math.floor(tf_coo_matrix.dense_shape[0]*.8)

In [15]:
# Create graph
# Placeholders for inputs (x), outputs(y)
with tf.variable_scope('Input'):
    x_original = tf.placeholder(tf.float32, shape=[None, num_features], name='X_original')
#     x_noisy = tf.placeholder(tf.float32, shape=[None, num_features], name='X_noisy')

# fc1 = fc_layer(x_noisy, h1, 'Hidden_layer', use_relu=True)
fc1 = fc_layer(x_original, h1, 'Hidden_layer', use_relu=True)
out = fc_layer(fc1, num_features, 'Output_layer', use_relu=False)

# Define the loss function, optimizer, and accuracy
with tf.variable_scope('Train'):
    with tf.variable_scope('Loss'):
        loss = tf.reduce_mean(tf.losses.mean_squared_error(x_original, out), name='loss')
        tf.summary.scalar('loss', loss)
    with tf.variable_scope('Optimizer'):
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, name='Adam-op').minimize(loss)

# Initializing the variables
init = tf.global_variables_initializer()

In [21]:
# # https://stackoverflow.com/a/40995666/2674061
    
# def next_batch(num, data):
#     '''
#     Return a total of `num` random samples
#     '''
#     idx = np.arange(0 , data.shape[0])
#     np.random.shuffle(idx)
#     idx = idx[:num]
# #     return data[idx].todense()
#     return data[idx]

In [16]:
# https://stackoverflow.com/a/40995666/2674061
def next_batch_data(batch_size):
    '''
    slice small number of data points from the large csr_matrix
    make a SparseTensor and return it 
    '''
    idx = np.arange(0 , data.shape[0])
    np.random.shuffle(idx)
    idx = idx[:batch_size]

    coo_matrix = data[idx].tocoo()
    tf_coo_matrix = tf.SparseTensorValue(
        indices=np.array([coo_matrix.row, coo_matrix.col]).T,
        values=coo_matrix.data,
        dense_shape=coo_matrix.shape)
    
    return tf.SparseTensor.from_value(tf_coo_matrix)

In [17]:
%%time
C = next_batch_data(5)

CPU times: user 53.8 ms, sys: 75.8 ms, total: 130 ms
Wall time: 129 ms


In [18]:
type(C)

tensorflow.python.framework.sparse_tensor.SparseTensor

In [19]:
C.get_shape()

TensorShape([Dimension(5), Dimension(23058100)])

### I need to skip the noise. The noise in general should have the chance to add values to zero elements as well. It is noise afterall and just adding it to the non-zero elements is not real!!!

#### prepare the test tensor

In [21]:
%%time
np.random.seed(10)
idx = np.arange(0 , test.shape[0])
np.random.shuffle(idx)
idx = idx[:200]

test_coo_matrix = test[idx].tocoo()
test_tf_coo_matrix = tf.SparseTensorValue(
    indices=np.array([test_coo_matrix.row, test_coo_matrix.col]).T,
    values=test_coo_matrix.data,
    dense_shape=test_coo_matrix.shape)

test_sp_tensor = tf.SparseTensor.from_value(test_tf_coo_matrix)

CPU times: user 2.08 s, sys: 2.34 s, total: 4.42 s
Wall time: 4.43 s


In [22]:
test_sp_tensor.get_shape()

TensorShape([Dimension(200), Dimension(23058100)])

In [23]:
# Launch the graph (session)
# sess = tf.InteractiveSession() # using InteractiveSession instead of Session to test network in separate cell
sess = tf.Session() # using InteractiveSession instead of Session to test network in separate cell
sess.run(init)
train_writer = tf.summary.FileWriter(logs_path, sess.graph)
num_tr_iter = int(num_train / batch_size)
global_step = 0
for epoch in range(epochs):
    print('Training epoch: {}'.format(epoch + 1))
    for iteration in range(num_tr_iter):
        batch_x = next_batch_data(batch_size)
        print(batch_x.get_shape())
#         batch_x_noisy = batch_x + noise_level * np.random.normal(loc=0.0, scale=1.0, size=batch_x.shape)

        global_step += 1
        print(global_step)
        # Run optimization op (backprop)
#         feed_dict_batch = {x_original: batch_x, x_noisy: batch_x_noisy}
        feed_dict_batch = {x_original: batch_x}
        _, summary_tr = sess.run([optimizer, merged], feed_dict=feed_dict_batch)
        train_writer.add_summary(summary_tr, global_step)

        if iteration % display_freq == 0:
            # Calculate and display the batch loss and accuracy
            loss_batch = sess.run(loss,
                                  feed_dict=feed_dict_batch)
            print("iter {0:3d}:\t Reconstruction loss={1:.3f}".
                  format(iteration, loss_batch))

    # Run validation after every epoch
#     x_valid_original  = mnist.validation.images
    x_valid_original  = test_sp_tensor
#     x_valid_noisy = x_valid_original + noise_level * np.random.normal(loc=0.0, scale=1.0, size=x_valid_original.shape)

    feed_dict_valid = {x_original: x_valid_original}
    loss_valid = sess.run(loss, feed_dict=feed_dict_valid)
    print('---------------------------------------------------------')
    print("Epoch: {0}, validation loss: {1:.3f}".
          format(epoch + 1, loss_valid))
    print('---------------------------------------------------------')
 

InvalidArgumentError: Cannot parse tensor from proto: dtype: DT_FLOAT
tensor_shape {
  dim {
    size: 23058100
  }
  dim {
    size: 1000
  }
}
float_val: 0

	 [[Node: Train/Optimizer/Hidden_layer/W_Hidden_layer/Adam-op/Initializer/zeros = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [23058100,1000] values: [0 0 0]...>, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op 'Train/Optimizer/Hidden_layer/W_Hidden_layer/Adam-op/Initializer/zeros', defined at:
  File "/home/esadrfa/libs/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/esadrfa/libs/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/esadrfa/libs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/esadrfa/libs/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.